In [1]:
# All vid gen techniques are described in
# https://huggingface.co/docs/diffusers/main/en/using-diffusers/text-img2vid
# https://huggingface.co/docs/diffusers/v0.31.0/using-diffusers/text-img2vid

# https://huggingface.co/tasks/text-to-image
# https://huggingface.co/black-forest-labs/FLUX.1-dev
# https://huggingface.co/docs/diffusers/using-diffusers/conditional_image_generation
# https://huggingface.co/blog/if
# https://huggingface.co/docs/diffusers/en/api/pipelines/stable_diffusion/text2img
# https://huggingface.co/docs/diffusers/v0.16.0/en/api/pipelines/stable_diffusion/text2img

# clone our repo and install requirements in colab
%cd /content/
!rm -rf LSVD
!git clone -b Inference https://github.com/davidg-h/LSVD.git
%cd LSVD
%pip install -r requirements.txt
!apt install ffmpeg
!git pull origin Inference

/content
Cloning into 'LSVD'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 141 (delta 51), reused 109 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 6.57 MiB | 12.41 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/LSVD
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
From https://github.com/davidg-h/LSVD
 * branch            Inference  -> FETCH_HEAD
Already up to date.


In [2]:
fps = 7
num_frames = 49 #49 / 7fps = ~6s
decode_chunk_size=2
prompt = "A happy woman walking down an aile in a convenience store, because she finally bought her favorite chocolate ice cream flavor."
negt = "bad quality, worse quality, low resolution"

In [3]:
# Text to Image Gen

import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler

model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.enable_model_cpu_offload()

image = pipe(prompt).images[0]

image.save("reference.png")

# cleanup
pipe.to("cpu")
del pipe
torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [11]:
# Img to Video Stable Video Diffusion

import torch
from diffusers import StableVideoDiffusionPipeline
from diffusers.utils import load_image, export_to_video

pipe = StableVideoDiffusionPipeline.from_pretrained(
    "stabilityai/stable-video-diffusion-img2vid-xt", torch_dtype=torch.float16, variant="fp16"
)

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

image = load_image(
    "/content/LSVD/reference.png"
    #"https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
    ) #TODO change
image = image.resize((1024, 576))

generator = torch.manual_seed(42)
try:
  frames = pipe(image, num_frames=num_frames, decode_chunk_size=8, generator=generator).frames[0]
  export_to_video(frames, "svd.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup
  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

# Worked in colab (stable; quality is mediocre --> movement aliasing)

model_index.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

vae/config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/196M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/3.05G [00:00<?, ?B/s]

...

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [4]:
# Img to Video AnimateDiff

import torch
from diffusers import AnimateDiffPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_video

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)

pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
scheduler = DDIMScheduler.from_pretrained(
    "emilianJR/epiCRealism",
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1,
)
pipe.scheduler = scheduler
pipe.enable_vae_slicing()

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

try:
  output = pipe(
      prompt=prompt,
      negative_prompt=negt,
      num_frames=32,
      decode_chunk_size=decode_chunk_size,
      guidance_scale=7.5,
      num_inference_steps=50,
      generator=torch.Generator("cpu").manual_seed(49),
  )
  frames = output.frames[0]
  export_to_video(frames, "animateDiff.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

# Worked in colab (stable on new runtime --> not really gen content of prompt)

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

In [5]:
# Text to Video ModelscopeT2V

import torch
from diffusers import DiffusionPipeline
from diffusers.utils import export_to_video

pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.enable_vae_slicing()

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

try:
  video_frames = pipe(
      prompt,
      num_frames=num_frames,
      ).frames[0]
  export_to_video(video_frames, "modelscopet2v.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

# Worked on Colab -> quality is not so good

model_index.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

model.fp16.safetensors:   0%|          | 0.00/681M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/2.82G [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [6]:
# Text/Img to Video I2VGen

import torch
from diffusers import I2VGenXLPipeline
from diffusers.utils import export_to_video, load_image

pipe = I2VGenXLPipeline.from_pretrained(
    "ali-vilab/i2vgen-xl", torch_dtype=torch.float16, variant="fp16"
)

# Optimize
pipe.enable_model_cpu_offload()
pipe.unet.enable_forward_chunking()

image_url = (
    "/content/LSVD/reference.png"
    #"https://huggingface.co/datasets/diffusers/docs-images/resolve/main/i2vgen_xl_images/img_0009.png" #TODO change
)
image = load_image(image_url).convert("RGB")

generator = torch.manual_seed(8888)

try:
  frames = pipe(
      prompt=prompt,
      image=image,
      num_frames=num_frames,
      decode_chunk_size=decode_chunk_size,
      num_inference_steps=50,
      negative_prompt=negt,
      guidance_scale=9.0,
      generator=generator,
  ).frames[0]
  export_to_video(frames, "i2v.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

model_index.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model.fp16.safetensors:   0%|          | 0.00/706M [00:00<?, ?B/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/705 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Exception: CUDA out of memory. Tried to allocate 1.32 GiB. GPU 0 has a total capacity of 14.75 GiB of which 235.06 MiB is free. Process 150420 has 14.52 GiB memory in use. Of the allocated memory 14.25 GiB is allocated by PyTorch, and 136.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
# Text/Img to Video CogVideoX

import torch
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

image = load_image(image="/content/LSVD/reference.png") #TODO change
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX-5b-I2V",
    torch_dtype=torch.bfloat16
)

pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

# Optimize
pipe.enable_model_cpu_offload()
#pipe.unet.enable_forward_chunking()

try:
  video = pipe(
      prompt=prompt,
      image=image,
      #decode_chunk_size=decode_chunk_size,
      num_videos_per_prompt=1,
      num_inference_steps=50,
      num_frames=num_frames,
      guidance_scale=6,
      generator=torch.Generator(device="cuda").manual_seed(42),
  ).frames[0]

  export_to_video(video, "cogVidX.mp4", fps=fps)
except Exception as e:
  raise Exception(e)
finally:
  # cleanup

  pipe.to("cpu")
  del pipe
  torch.cuda.empty_cache()

model_index.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer/added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/839 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Exception: CUDA out of memory. Tried to allocate 80.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 35.06 MiB is free. Process 150420 has 14.71 GiB memory in use. Of the allocated memory 14.47 GiB is allocated by PyTorch, and 110.59 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)